In [ ]:
%matplotlib inline
import pandas as pd
from rich import print
import numpy as np
import matplotlib.pyplot as plt
#from quickbin import bin2d
from scipy.stats import binned_statistic_2d
from pyarrow import ArrowInvalid

In [ ]:
nd_files = !ls data/e*/*nd*catalog*
fd_files = !ls data/e*/*fd*catalog*
print(len(nd_files))

In [ ]:
tbl = pd.read_parquet(nd_files[-1])
#print(len(tbl['NUV_MDL_CPS']))
#print(tbl.shape)
tbl = pd.DataFrame({})
for f in nd_files:
    try:
        tbl = pd.concat([tbl,pd.read_parquet(f)],ignore_index=True,join='outer',axis=0,copy=False)
    except ArrowInvalid:
        print(f'Unable to open {f}')
#print(tbl.shape)

In [ ]:
aper_photom = {'cps':tbl['NUV_CPS_A4'],
               'bg':(9**2)*(tbl['NUV_CPS_A6']-tbl['NUV_CPS_A5'])/(17.3**2-12.8**2),
              }
modl_photom = {'cps':tbl['NUV_MDL_CPS'],
               'bg':tbl['NUV_MDL_BKG_CPS']
              }

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(aper_photom['cps']-aper_photom['bg'],
         modl_photom['cps'],'k.',alpha=0.4)
plt.xlim([0,500]);plt.ylim([0,500]);
plt.plot([0,500],[0,500],'k:',alpha=0.5)
plt.xlabel('aperture photometry (aper4 cps-bg)')
plt.ylabel('model photometry (cps)')

plt.figure(figsize=(10,10))
plt.plot(aper_photom['cps']-aper_photom['bg'],
         modl_photom['cps'],'k.',alpha=0.4)
plt.xlim([0,50]);plt.ylim([0,50]);
plt.plot([0,500],[0,500],'k:',alpha=0.5)
plt.xlabel('aperture photometry')
plt.ylabel('model photometry')

plt.figure(figsize=(10,10))
plt.plot(aper_photom['cps']-aper_photom['bg'],
         modl_photom['cps'],'k.',alpha=0.4)
plt.xlim([0,10]);plt.ylim([0,10]);
plt.plot([0,50],[0,50],'k:',alpha=0.3)
plt.xlabel('aperture photometry')
plt.ylabel('model photometry')

sigmas = tbl['NUV_MDL_SIGMA']
ix = np.where((np.isfinite(sigmas)) & 
              (sigmas<6) & (sigmas>0.5) &
              (tbl['NUV_SOFTEDGE_FLAG_A6']==0) & (tbl['NUV_HARDEDGE_FLAG_A6']==0) &
              (tbl['NUV_GHOST_FLAG_A6']==0) & (tbl['NUV_HOTSPOT_FLAG_A6']==0))
print(f'{100*np.round(len(ix[0])/len(sigmas),3)}% sources retained')

#plt.figure(figsize=(10,10))
plt.plot((aper_photom['cps']-aper_photom['bg']).iloc[ix],
         modl_photom['cps'].iloc[ix],
         'b.',alpha=0.1)
plt.xlim([0,10]);plt.ylim([0,10]);
plt.plot([0,50],[0,50],'k:',alpha=1)
plt.xlabel('aperture photometry (aper4 cps-bg)')
plt.ylabel('model photometry (cps)')

print(len(ix[0]))
print(len(sigmas))

In [ ]:
plt.hist(tbl['NUV_MDL_SIGMA'],range=[0,10],bins=100);
plt.xlabel('SIGMA (arcsec)')
plt.title(f'n={len(np.where(np.isfinite(tbl['NUV_MDL_SIGMA']))[0])}')

In [ ]:
aper_radii = [1.5, 2.3, 3.8, 6.0, 9.0, 12.8, 17.3]
for aper,apcorr in enumerate([1.33, 0.62, 0.21, 0.12, 0.08,]):# 0.06, 0.04]):
    aper_photom = {'cps':tbl[f'NUV_CPS_A{aper}'],
                   'bg':(aper_radii[aper]**2)*(tbl['NUV_CPS_A6']-tbl['NUV_CPS_A5'])/(17.3**2-12.8**2),
                  }
    modl_photom = {'cps':tbl['NUV_MDL_CPS'],
                   'bg':tbl['NUV_MDL_BKG_CPS']
                  }
    
    plt.figure(figsize=(10,2))
    plt.title(f'APER{aper}')
    plt.plot(
        #(1+apcorr)*
             (aper_photom['cps']-aper_photom['bg']).iloc[ix],
             modl_photom['cps'].iloc[ix],
             'b.',alpha=0.4)
    plt.xlim([0,10]);plt.ylim([0,10]);
    plt.plot([0,50],[0,50],'k:',alpha=0.5)
    plt.xlabel('aperture photometry (cps-bg)')
    plt.ylabel('model photometry (cps)')


In [ ]:
plt.figure()
plt.plot(tbl['NUV_MDL_SIGMA'],tbl['NUV_MDL_CPS'],'k.',alpha=0.4)
plt.xlim([0,10])
plt.ylim([0,50])
plt.xlabel('SIGMA')
plt.ylabel('CPS')

plt.figure()
plt.plot(tbl['NUV_MDL_SIGMA'],tbl['NUV_MDL_CPS'],'k.',alpha=0.4);
plt.xlim([0,10])
plt.xlabel('SIGMA')
plt.ylabel('CPS')

In [ ]:
plt.hist(tbl['NUV_MDL_SIGMA'],range=[0,10],weights=tbl['NUV_MDL_CPS'],bins=50);
plt.xlabel('SIGMA')
plt.title('weighted by brightness')
#plt.plot(tbl['NUV_MDL_SIGMA'],tbl['NUV_MDL_CPS'],'k.',alpha=0.4);

In [ ]:
data = pd.DataFrame({})
for f in nd_files[:-1]:
    try:
        data = pd.concat([data,pd.read_parquet(f)],ignore_index=True,join='outer',axis=0,copy=False)
    except ArrowInvalid:
        print(f'Unable to open {f}')
data

In [ ]:
plt.plot(aper_photom['bg'],tbl['NUV_MDL_BKG_CPS']*(9**2),'k.',alpha=0.2)
plt.xlabel('annulus bg (aper4 cps)')
plt.ylabel('model bg (aper4 cps)')

plt.figure()
plt.title('model bg / aper bg')
plt.hist(tbl['NUV_MDL_BKG_CPS']*(9**2) / aper_photom['bg'],bins=100);

In [ ]:
tbl['RA'].max()

In [ ]:

binned_statistic_2d(tbl['RA'].values,tbl['DEC'].values,tbl['NUV_MDL_BKG_CPS'],bins=10)

In [ ]:
?binned_statistic_2d